In [1]:
import arviz as az
import numpy as np
import pymc as pm
from pymc.math import dot, stack, concatenate, exp, invlogit, logit

%load_ext lab_black
%load_ext watermark

# Rats

This example goes further into dealing with missing data in PyMC, including in the predictor variables.

Adapted from [Unit 8: ratsignorable1.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit8/ratsignorable1.odc), [ratsignorable2.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit8/ratsignorable2.odc), and [ratsinformative.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit8/ratsinformative.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/rats.txt).

Associated lecture videos: Unit 8 lesson 2

## Problem statement

We had a previous example about [Dugongs](https://areding.github.io/6420-pymc/Unit6-dugongs.html) that dealt with missing data in the observed data (y values). This example shows how to deal with missing data in the input data (x). It's still pretty easy. You could look at it like creating another likelihood in the model, a very simple one where the observed data is x, and you use a single distribution to fill in the missing values (see ```x_imputed``` in the model below).

Original paper [here.](https://www.jstor.org/stable/pdf/2289594.pdf)

Gelfand et al 1990 consider the problem of missing data, and delete the last observation of cases 6-10, the last two from 11-20, the last 3 from 21-25 and the last 4 from 26-30.  The appropriate data file is obtained by simply replacing data values by NA (see below). The model specification is unchanged, since the distinction between observed and unobserved quantities is made in the data file and not the model specification.

In [2]:
x = np.array([8.0, 15.0, 22.0, 29.0, 36.0])

In [3]:
# import y data and create mask (missing data is represented as nan in the file)
y = np.loadtxt("../data/rats.txt")
y = np.nan_to_num(y, nan=-1)  # nan to -1
y = np.ma.masked_values(y, value=-1)  # create mask

## Model 1

This first model we only have missing data in our response variable (y). Notice that I made the shapes of alpha and beta (30, 1) instead of just 30. This is so that they broadcast correctly when combined (```mu = alpha + beta * x```). The NumPy docs have a helpful [page about broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html).


In [4]:
with pm.Model() as m:
    alpha_c = pm.Normal("alpha_c", 0, tau=1e-6)
    alpha_tau = pm.Gamma("alpha_tau", 0.001, 0.001)
    beta_c = pm.Normal("beta_c", 0, tau=1e-6)
    beta_tau = pm.Gamma("beta_tau", 0.001, 0.001)

    alpha = pm.Normal(
        "alpha", alpha_c, tau=alpha_tau, shape=(30, 1)
    )  # (30, 1) for broadcasting
    beta = pm.Normal("beta", beta_c, tau=beta_tau, shape=(30, 1))
    lik_tau = pm.Gamma("lik_tau", 0.001, 0.001)
    sigma = pm.Deterministic("sigma", 1 / lik_tau**0.5)

    mu = alpha + beta * x

    pm.Normal("likelihood", mu, tau=lik_tau, observed=y)

    trace = pm.sample(
        5000, tune=1000, cores=4, init="jitter+adapt_diag_grad", target_accept=0.9
    )

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/pymc/model.py:1431: ImputationWarning: Data in likelihood contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag_grad...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_c, alpha_tau, beta_c, beta_tau, alpha, beta, lik_tau, likelihood_missing]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 153 seconds.
There were 14 divergences after tuning. Increase `target_accept` or reparameterize.


In [5]:
az.summary(
    trace,
    hdi_prob=0.95,
    var_names=["alpha_c", "alpha_tau", "beta_c", "beta_tau", "sigma"],
    kind="stats",
)

,mean,sd,hdi_2.5%,hdi_97.5%
alpha_c,101.237,2.297,96.749,105.752
alpha_tau,0.034,0.151,0.003,0.052
beta_c,6.566,0.164,6.244,6.880
beta_tau,2.884,1.245,0.966,5.276
sigma,6.031,0.680,4.761,7.381


## Model 2: Imputing missing predictor variable data

This is the same model, except we now have missing x data.

In [22]:
x_miss = np.array([8.0, 15.0, 22.0, -1, 36.0])
x_miss = np.ma.masked_values(x_miss, value=-1)

In [23]:
x_miss

masked_array(data=[8.0, 15.0, 22.0, --, 36.0],
             mask=[False, False, False,  True, False],
       fill_value=-1.0)

In [25]:
with pm.Model() as m:
    alpha_c = pm.Normal("alpha_c", 0, tau=1e-6)
    alpha_tau = pm.Gamma("alpha_tau", 0.001, 0.001)
    beta_c = pm.Normal("beta_c", 0, tau=1e-6)
    beta_tau = pm.Gamma("beta_tau", 0.001, 0.001)

    alpha = pm.Normal("alpha", alpha_c, tau=alpha_tau, shape=(30, 1))
    beta = pm.Normal("beta", beta_c, tau=beta_tau, shape=(30, 1))
    lik_tau = pm.Gamma("lik_tau", 0.001, 0.001)
    sigma = pm.Deterministic("sigma", 1 / lik_tau**0.5)

    x_imputed = pm.TruncatedNormal(
        "x_imputed", mu=20, sigma=10, lower=0, observed=x_miss
    )

    mu = alpha + beta * x_imputed

    pm.Normal("likelihood", mu, tau=lik_tau, observed=y)

    trace_2 = pm.sample(
        5000, tune=1000, cores=4, init="jitter+adapt_diag_grad", target_accept=0.9
    )

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/pymc/model.py:1431: ImputationWarning: Data in x_imputed contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/pymc/model.py:1431: ImputationWarning: Data in likelihood contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Auto-assigning NUTS sampler...
INFO:pymc:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag_grad...
INFO:pymc:Initializing NUTS using jitter+adapt_diag_grad...
Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc:Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha_c, alpha_tau, beta_c, beta_tau, alpha, beta, lik_tau, x_imputed_missing, likelihood_missing]
INFO:pymc:NUTS: [alpha_c, alpha_tau, beta_c, beta_tau, alpha, beta, lik_tau, x_impute

Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 117 seconds.
INFO:pymc:Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 117 seconds.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
ERROR:pymc:There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
There were 4 divergences after tuning. Increase `target_accept` or reparameterize.
ERROR:pymc:There were 4 divergences after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
ERROR:pymc:There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


In [26]:
az.summary(
    trace_2,
    hdi_prob=0.95,
    var_names=["~alpha", "~beta", "~likelihood_missing"],
    kind="stats",
)

,mean,sd,hdi_2.5%,hdi_97.5%
alpha_c,101.876,2.358,97.235,106.482
beta_c,6.509,0.167,6.179,6.833
alpha_tau,0.021,0.052,0.004,0.042
beta_tau,3.001,1.278,0.932,5.438
lik_tau,0.029,0.006,0.017,0.041
x_imputed_missing[0],29.500,0.385,28.754,30.267
sigma,5.970,0.654,4.791,7.307


## Model 3: Non-ignorable missingness

Probability of missingness increases approx. at a rate of 1% with increasing the weight.

In [45]:
y = np.atleast_2d(np.array([177.0, 236.0, 285.0, 350.0, -1]))  # original value was 320
y = np.ma.masked_values(y, value=-1)  # create masked array
# y.mask is equivalent to the "miss" array from the professor's example
miss = y.mask
x = np.array([8.0, 15.0, 22.0, 29.0, 36.0])

In [76]:
t = 0.1
s = 1 / t  # convert BUGS dlogis tau to s for pymc
b = np.log(1.01)

with pm.Model() as m:
    a = pm.Logistic("a", mu=0, s=s)
    alpha = pm.Flat("alpha")
    beta = pm.Flat("beta")
    log_sigma = pm.Flat("log_sigma")
    sigma2 = pm.Deterministic("sigma2", exp(2 * log_sigma))
    tau = pm.Deterministic("tau", 1 / sigma2)

    mu = pm.Deterministic("mu", alpha + beta * x)
    y_imputed = pm.Normal("likelihood", mu, tau=tau, observed=y)

    p = pm.Deterministic("p", invlogit(a + b * y_imputed))
    pm.Bernoulli("missing", p=p, observed=miss)

    trace_3 = pm.sample(4000, tune=2000, init="jitter+adapt_diag_grad", target_accept=.7)

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/pymc/model.py:1431: ImputationWarning: Data in likelihood contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
Auto-assigning NUTS sampler...
INFO:pymc:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag_grad...
INFO:pymc:Initializing NUTS using jitter+adapt_diag_grad...
Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc:Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, alpha, beta, log_sigma, likelihood_missing]
INFO:pymc:NUTS: [a, alpha, beta, log_sigma, likelihood_missing]


Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 22 seconds.
INFO:pymc:Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 22 seconds.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
ERROR:pymc:There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
There were 168 divergences after tuning. Increase `target_accept` or reparameterize.
ERROR:pymc:There were 168 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.5763, but should be close to 0.8. Try to increase the number of tuning steps.
There were 86 divergences after tuning. Increase `target_accept` or reparameterize.
ERROR:pymc:There were 86 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6707, but should be close to 0.8. Try to 

In [79]:
az.summary(trace_3, hdi_prob=0.95, kind="stats")

,mean,sd,hdi_2.5%,hdi_97.5%
a,-4.861,1.447,-7.748,-2.315
alpha,111.743,37.921,81.477,139.499
beta,8.173,1.543,6.797,9.617
log_sigma,1.857,0.708,0.776,3.238
likelihood_missing[0],408.372,52.684,367.884,440.877
sigma2,1261.074,31508.011,3.312,569.541
tau,0.046,0.045,0.000,0.133
mu[0],177.124,29.071,155.395,193.343
mu[1],234.333,24.006,220.789,244.858
mu[2],291.542,23.254,279.611,304.347


In [78]:
%watermark -n -u -v -iv -p aesara,aeppl

Last updated: Sun Jul 31 2022

Python implementation: CPython
Python version       : 3.10.5
IPython version      : 8.4.0

aesara: 2.7.7
aeppl : 0.0.32

pymc : 4.1.3
numpy: 1.23.1
arviz: 0.12.1

